### Define the Python Environment

In [ ]:
# Load the Python environment.
import matplotlib.pyplot as plt
import numpy
import tcdiags
from exceptions import TropCycMPIError
from ioapps import netcdf4_interface
from mpl_toolkits.basemap import Basemap
from tools import parser_interface

## Jupyter Notebook: tcmpi.ipynb

Email: henry.winterbottom@noaa.gov

This program is free software: you can redistribute it and/or modify
it under the terms of the respective public license published by the
Free Software Foundation and included with the repository within
which this application is contained.

This program is distributed in the hope that it will be useful, but
WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

---

## Description

<br>This Jupyter notebook provides an application to compute and plot tropical cyclone (TC) maximum potential intensity (MPI) as decribed in Bister and Emanuel.</br>

## Author

<br>Henry R. Winterbottom; 14 March 2023</br>

---

<b>Licensing:</b> GNU Lesser General Public License v2.1<br>
<b>Maintainer:</b> Henry R. Winterbottom (henry.winterbottom@noaa.gov)<br>
<b>History:</b><br>
<ul>
    <li>2023-03-14: Henry Winterbottom -- Initial implementation.</li>
</ul>

---

### User Configuration

In [ ]:
# Define the path to the YAML-formatted TC-diagnostics configuration file.
yaml_file = "/Users/henry.winterbottom/trunk/UFS/ufs_tcdiags/parm/tcdiags.notebook.yaml"

# Define the TC MPI metric plotting attributes.
import cmocean
mslp_cint = 25.0
mslp_cmax = 1025.0
mslp_cmin = 850.0
mslp_cmap = cmocean.cm.haline

vmax_cint = 20.0
vmax_cmax = 120.0
vmax_cmin = 0.0
vmax_cmap = "jet"

### Compute the Tropical Cyclone Maximum Potential Intensity

In [ ]:
# Define the configuration object attributes.
config_obj = parser_interface.object_define()
config_obj = parser_interface.object_setattr(
    object_in=config_obj, key="yaml_file", value=yaml_file)
config_obj = parser_interface.object_setattr(
    object_in=config_obj, key="tcmpi", value="True")

# Compute the tropical cyclone (TC) maximum potential intensity (MPI) metrics.
tcmpi = tcdiags.TCDiags(options_obj=config_obj)
tcmpi_out = parser_interface.dict_key_value(
    dict_in=tcmpi.yaml_dict["tcmpi"], key="output_file", force=True,
    no_split=True)
if tcmpi_out is None:
    msg = ("The attribute `output_file` could not be determined for the "
           f"{parse_interface.dict_key_value(dict_in=tcmpi.yaml_dict,key='tcmpi')}."
           "Aborting!!!"
          )
    raise TropCycMPIError(msg=msg)
        
tcmpi.run()

### Build the Basemap Attributes

In [ ]:
def build_basemap():
    
    # Define the basemap attributes and return.
    basemap = Basemap(resolution="c", projection="cyl",
                     llcrnrlat=-90.0, llcrnrlon=0.0,
                     urcrnrlat=90.0, urcrnrlon=360.0)
    (x, y) = numpy.meshgrid(lon, lat)
    
    basemap.drawcoastlines(color="lightgray", linewidth=0.25)
    basemap.drawmeridians(color="black", meridians=numpy.arange(0.0, 360.01, 30.0),
                          labels=[False, False, True, True],linewidth=0.1, dashes=[10, 10], 
                          fontsize=8)
    basemap.drawparallels(color="black", circles=numpy.arange(-80.0, 80.01, 40.0),
                          labels=[True, True, False, False], linewidth=0.1, dashes=[10,10], 
                          fontsize=8)
    basemap.fillcontinents(color="lightgray")
    
    return (basemap, x,y)

### Plot the Tropical Cyclone Maximum Potential Intensity Metrics

In [ ]:
# Read in the grid projection attributes; scale accordingly.
(lat, lon) = [netcdf4_interface.ncreadvar(ncfile=tcmpi_out,
                                         ncvarname=varname) for varname in ["lat", "lon"]]
lon = lon + 360.0

# Read in the TC MPI metrics.
mslp = netcdf4_interface.ncreadvar(ncfile=tcmpi_out,
                                  ncvarname="mslp")
vmax = netcdf4_interface.ncreadvar(ncfile=tcmpi_out,
                                  ncvarname="vmax")

# Plot the TC MPI minimum sea-level pressure metric.
levels = numpy.linspace(mslp_cmin, mslp_cmax, 255)
(basemap, x, y) = build_basemap()
basemap.contourf(x, y, mslp/100.0, levels=levels, cmap=mslp_cmap)
ticks = numpy.arange(mslp_cmin, (mslp_cmax + 0.01), mslp_cint)
plt.colorbar(orientation="horizontal", ticks=ticks, pad=0.1,
            aspect=50, label="Maximum Potential Intensity Sea-Level Pressure (hPa)")
plt.savefig("tcmpi.mslp.png", dpi=500, transparent=True, bbox_inches="tight")
plt.show()   

# Plot the TC MPI maximum wind speed metric.
levels = numpy.linspace(vmax_cmin, vmax_cmax, 255)
(basemap, x, y) = build_basemap()
basemap.contourf(x, y, vmax, levels=levels, cmap=vmax_cmap)
ticks = numpy.arange(vmax_cmin, (vmax_cmax + 0.01), vmax_cint)
plt.colorbar(orientation="horizontal", ticks=ticks, pad=0.1,
            aspect=50, label="Maximum Potential Intensity Wind Speed (ms$^{-1}$)")
plt.savefig("tcmpi.vmax.png", dpi=500, transparent=True, bbox_inches="tight")
plt.show()